In [9]:
#Combining all offensive, defensive, and special_teams files into one file for each
def add_prefix_to_columns(df, prefix):
    return df.rename(columns=lambda x: f"{prefix}_{x}" if x != 'Team' else x)

def merge_stats(file_list, category_prefix, input_dir, output_dir):
    merged_stats = pd.DataFrame()
    for file_name in file_list:
        
        prefix = f"{category_prefix}_{file_name.split('Stats')[0].lower()[:2]}"
        
        stats_path = os.path.join(input_dir, file_name)
        current_stats = pd.read_csv(stats_path)
        
        current_stats['Team'] = current_stats['Team'].str.strip()

        current_stats = add_prefix_to_columns(current_stats, prefix)
        
        if merged_stats.empty:
            merged_stats = current_stats
        else:
            merged_stats = pd.merge(merged_stats, current_stats, on='Team', how='outer')
    
    output_filepath = os.path.join(output_dir, f"{category_prefix}_merged_stats.csv")
    merged_stats.to_csv(output_filepath, index=False)
    print(f"{category_prefix.title()} merged stats saved to: {output_filepath}")

input_base_path = '/Users/charlesmorgan/Desktop/NFL Stats/'
output_base_path = '/Users/charlesmorgan/Desktop/NFL Stats/'    

offensive_files = [
    'PassingStatsOffense.csv', 'RushingStatsOffense.csv',
    'RecievingStatsOffense.csv', 'DownsStatsOffense.csv',
    'ScoringStatsOffense.csv'
]
defensive_files = [
    'PassingStatsDefense.csv', 'RushingStatsDefense.csv', 
    'RecievingStatsDefense.csv', 'InterceptionStatsDefense.csv',
    'FumbleStatsDefense.csv', 'DownsStatsDefense.csv', 'TackleStatsDefense.csv',
    'ScoringStatsDefense.csv'
]
special_teams_files = [
    'FeildGoalStatsSpecialTeams.csv',
    'KickoffReturnsStatsSpecialTeams.csv', 'PuntReturnsStatsSpecialTeams.csv',
    'PuntingStatsSpecialTeams.csv', 'KickoffsSpecialTeams.csv', 
    'ScoringStatsSpecialTeams.csv'
]

merge_stats(offensive_files, 'offense', input_base_path, output_base_path)
merge_stats(defensive_files, 'defense', input_base_path, output_base_path)
merge_stats(special_teams_files, 'specialteams', input_base_path, output_base_path)

Offense merged stats saved to: /Users/charlesmorgan/Desktop/NFL Stats/offense_merged_stats.csv
Defense merged stats saved to: /Users/charlesmorgan/Desktop/NFL Stats/defense_merged_stats.csv
Specialteams merged stats saved to: /Users/charlesmorgan/Desktop/NFL Stats/specialteams_merged_stats.csv


In [10]:
#Combining the offensive, defensive, and special_teams files into one big file that includes all stats
import pandas as pd
import os

offense_path = os.path.join(output_base_path, 'offense_merged_stats.csv')
defense_path = os.path.join(output_base_path, 'defense_merged_stats.csv')
special_teams_path = os.path.join(output_base_path, 'specialteams_merged_stats.csv')

offense_stats = pd.read_csv(offense_path)
defense_stats = pd.read_csv(defense_path)
special_teams_stats = pd.read_csv(special_teams_path)

combined_stats = pd.merge(offense_stats, defense_stats, on='Team', how='outer')

combined_stats = pd.merge(combined_stats, special_teams_stats, on='Team', how='outer')

combined_stats_path = os.path.join(output_base_path, 'combined_all_stats.csv')
combined_stats.to_csv(combined_stats_path, index=False)

print(f"All stats merged and saved to: {combined_stats_path}")

All stats merged and saved to: /Users/charlesmorgan/Desktop/NFL Stats/combined_all_stats.csv


In [11]:
#Adding the stats of each team in each game, along with a column indicating if the home team won or loss, so
#the model has a column to predict on.

import pandas as pd
import os

file_names = [
    'offense_merged_stats.csv', 'defense_merged_stats.csv', 'specialteams_merged_stats.csv',
    'combined_all_stats.csv', 'FeildGoalStatsSpecialTeams.csv', 'PassingStatsDefense.csv', 'RushingStatsDefense.csv', 
    'RecievingStatsDefense.csv', 'PassingStatsOffense.csv', 'InterceptionStatsDefense.csv',
    'KickoffReturnsStatsSpecialTeams.csv', 'PuntReturnsStatsSpecialTeams.csv',
    'RushingStatsOffense.csv', 'RecievingStatsOffense.csv',
    'PuntingStatsSpecialTeams.csv', 'KickoffsSpecialTeams.csv', 
    'ScoringStatsSpecialTeams.csv', 'DownsStatsOffense.csv', 'ScoringStatsOffense.csv',
    'FumbleStatsDefense.csv', 'DownsStatsDefense.csv', 'TackleStatsDefense.csv',
    'ScoringStatsDefense.csv']

input_base_path = '/Users/charlesmorgan/Desktop/NFL Stats/'
output_base_path = '/Users/charlesmorgan/Desktop/Merged Data/'  

for file in file_names:
    stats_filepath = os.path.join(input_base_path, file)
    output_filepath = os.path.join(output_base_path, file)
    
    team_stats = pd.read_csv(stats_filepath)
    games = pd.read_csv('/Users/charlesmorgan/Desktop/NFL Stats/Games.csv')
    
    team_stats['Team'] = team_stats['Team'].str.strip()
    games['Home Team'] = games['Home Team'].str.strip()
    games['Away Team'] = games['Away Team'].str.strip()

    merged_data = pd.merge(games, team_stats, left_on='Home Team', right_on='Team', suffixes=('', '_home'))

    merged_data = pd.merge(merged_data, team_stats, left_on='Away Team', right_on='Team', suffixes=('_home', '_away'))

    merged_data.drop(['Team_home', 'Team_away'], axis=1, inplace=True)

    merged_data['Home_Win'] = (merged_data['Home Team Score'] > merged_data['Away Team Score']).astype(int)
    merged_data.drop(['Home Team Score', 'Away Team Score'], axis=1, inplace=True)

    merged_data.to_csv(output_filepath, index=False)

    print("Merged dataset saved to:", output_filepath)

Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/offense_merged_stats.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/defense_merged_stats.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/specialteams_merged_stats.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/combined_all_stats.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/FeildGoalStatsSpecialTeams.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/PassingStatsDefense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/RushingStatsDefense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/RecievingStatsDefense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/PassingStatsOffense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/InterceptionStatsDefense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/KickoffReturn

/var/folders/57/xt02n_1159z1j0k3c7_r0vyh0000gn/T/ipykernel_23294/6705706.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data['Home_Win'] = (merged_data['Home Team Score'] > merged_data['Away Team Score']).astype(int)


Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/KickoffsSpecialTeams.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/ScoringStatsSpecialTeams.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/DownsStatsOffense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/ScoringStatsOffense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/FumbleStatsDefense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/DownsStatsDefense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/TackleStatsDefense.csv
Merged dataset saved to: /Users/charlesmorgan/Desktop/Merged Data/ScoringStatsDefense.csv
